In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

conf = SparkConf().setAppName("").setMaster("spark://workspace:7077")
sparkContext = SparkContext(conf=conf)
spark = SparkSession(sparkContext=sparkContext)

23/12/30 16:45:19 WARN Utils: Your hostname, workspace resolves to a loopback address: 127.0.0.1; using 220.118.158.128 instead (on interface eno1)
23/12/30 16:45:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/30 16:45:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.functions import expr, explode, col

In [3]:
json_dir = "file:///home/hooniegit/git/Spotify-DemoProject/fastapi/data/albums/2023-12-29"

In [15]:
df = spark.read.option("multiline", "true").json(json_dir)
df.printSchema()

root
 |-- albums: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- album_type: string (nullable = true)
 |    |    |-- artists: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- external_urls: struct (nullable = true)
 |    |    |    |    |    |-- spotify: string (nullable = true)
 |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |    |-- type: string (nullable = true)
 |    |    |    |    |-- uri: string (nullable = true)
 |    |    |-- copyrights: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- text: string (nullable = true)
 |    |    |    |    |-- type: string (nullable = true)
 |    |    |-- external_ids: struct (nullable = true)
 |    |    |    |-- amgid: string (nullable = true)
 |    |    |    |--

In [7]:
df = spark.read.option("multiline", "true").json(json_dir) \
    .select(explode('albums').alias('albums'))

df.show()

+--------------------+
|              albums|
+--------------------+
|{compilation, [{{...|
|{single, [{{https...|
|{single, [{{https...|
|{compilation, [{{...|
|{compilation, [{{...|
|{album, [{{https:...|
|{compilation, [{{...|
|{single, [{{https...|
|{album, [{{https:...|
|{compilation, [{{...|
|{single, [{{https...|
|{single, [{{https...|
|{album, [{{https:...|
|{single, [{{https...|
|{single, [{{https...|
|{album, [{{https:...|
|{compilation, [{{...|
|{single, [{{https...|
|{compilation, [{{...|
|{single, [{{https...|
+--------------------+
only showing top 20 rows



In [11]:
df = spark.read.option("multiline", "true").json(json_dir) \
    .select(explode('albums').alias('albums')) \
    .withColumn("image", explode(expr("albums.images"))) \
    .filter(col("image.height") == 640) \
    .withColumn("artist_id", explode(expr("albums.artists.id"))) \
    .withColumn("album_id", expr("albums.id")) \
    .withColumn("album_name", expr("albums.name")) \
    .withColumn("album_image_url", expr("image.url")) \
    .withColumn("track", explode(expr("albums.tracks.items"))) \
    .withColumn("track_id", expr("track.id")) \
    .withColumn("track_name", expr("track.name")) \
    .withColumn("duration_ms", expr("track.duration_ms")) \
    .withColumn("explicit", expr("track.explicit")) \
    .withColumn("disc_number", expr("track.disc_number")) \
    .withColumn("track_number", expr("track.track_number")) \
    .select("album_id", "album_name", "artist_id", "albums.album_type", "albums.label", "album_image_url", "track_id", "track_name", "track_number", "disc_number", "duration_ms", "explicit", "albums.release_date", "albums.total_tracks")

df.show()

+--------------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+------------+-----------+-----------+--------+------------+------------+
|            album_id|          album_name|           artist_id| album_type|               label|     album_image_url|            track_id|          track_name|track_number|disc_number|duration_ms|explicit|release_date|total_tracks|
+--------------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+------------+-----------+-----------+--------+------------+------------+
|6Bi5Q5VIzGz0c6zat...|Victoria's Secret...|0LyfQWJT6nXafLPZq...|compilation|Warner Music Grou...|https://i.scdn.co...|4FU8T0t7YASGlMaA8...|   Victoria's Secret|           1|          1|     176124|    true|  2022-09-16|          50|
|6Bi5Q5VIzGz0c6zat...|Victoria's Secret...|0LyfQWJT6nXafLPZq...|comp

In [13]:
df.head(60)

[Row(album_id='6Bi5Q5VIzGz0c6zatIVqLq', album_name="Victoria's Secret - Pop Bangers", artist_id='0LyfQWJT6nXafLPZqxe9Of', album_type='compilation', label='Warner Music Group - X5 Music Group', album_image_url='https://i.scdn.co/image/ab67616d0000b273f42e6a067b35a658b051cab0', track_id='4FU8T0t7YASGlMaA8uaJ2R', track_name="Victoria's Secret", track_number=1, disc_number=1, duration_ms=176124, explicit=True, release_date='2022-09-16', total_tracks=50),
 Row(album_id='6Bi5Q5VIzGz0c6zatIVqLq', album_name="Victoria's Secret - Pop Bangers", artist_id='0LyfQWJT6nXafLPZqxe9Of', album_type='compilation', label='Warner Music Group - X5 Music Group', album_image_url='https://i.scdn.co/image/ab67616d0000b273f42e6a067b35a658b051cab0', track_id='6QEv2OySmn8WAjTlHrTJBV', track_name='Left and Right', track_number=2, disc_number=1, duration_ms=154486, explicit=False, release_date='2022-09-16', total_tracks=50),
 Row(album_id='6Bi5Q5VIzGz0c6zatIVqLq', album_name="Victoria's Secret - Pop Bangers", artist

In [ ]:
spark.stop()